# Request Log Analysis

## Functionalities
- Display a summary of requests and their performance metrics.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    loadgen.tar.gz
  ...
  [node-n]/
    loadgen.tar.gz
```
A tarball `loadgen.tar.gz` contains a request log file named `loadgen.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-10-10-18-35-22"
# Ramp up duration (in sec)
RAMP_UP_DURATION = 60
# Ramp down duration (in sec)
RAMP_DOWN_DURATION = 60

########## LATENCY
# Max expected value
MAX_LATENCY_IN_S = 10
# Bin size
LATENCY_BIN_IN_MS = 50

## Notebook Setup

In [ ]:
# Import libraries.
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import tarfile
import warnings
warnings.filterwarnings("ignore")

# Constants
REQUEST_LOG_PATTERN = r"^\[(\d+\-\d+\-\d+ \d+:\d+:\d+.\d+)\] (.+) (.+) (\d+) - latency=(\d+.\d+)$"
URL_PATTERN = r"^http://[\w\.]+:\d+/{path}/?\??{qs}$"
REQUEST_TO_TYPE = {
    (URL_PATTERN.format(path="account/\d+", qs=""), "GET"): "retrieve_account",
    (URL_PATTERN.format(path="account", qs=""), "POST"): "create_account",
    (URL_PATTERN.format(path="account/\d+", qs=""), "PUT"): "update_account",
    (URL_PATTERN.format(path="follow", qs="followee_id=\d+"), "GET"): "retrieve_account_followers",
    (URL_PATTERN.format(path="follow", qs="follower_id=\d+"), "GET"): "retrieve_account_followees",
    (URL_PATTERN.format(path="follow", qs=""), "POST"): "follow_account",
    (URL_PATTERN.format(path="follow/\d+", qs=""), "DELETE"): "delete_follow",
    (URL_PATTERN.format(path="like", qs="account_id=\d+"), "GET"): "retrieve_account_likes",
    (URL_PATTERN.format(path="like", qs="post_id=\d+"), "GET"): "retrieve_post_likes",
    (URL_PATTERN.format(path="like", qs=""), "POST"): "like_post",
    (URL_PATTERN.format(path="like/\d+", qs=""), "DELETE"): "delete_like",
    (URL_PATTERN.format(path="post", qs=""), "GET"): "retrieve_recent_posts",
    (URL_PATTERN.format(path="post", qs="author_id=\d+"), "GET"): "retrieve_account_posts",
    (URL_PATTERN.format(path="post/\d+", qs=""), "GET"): "retrieve_post",
    (URL_PATTERN.format(path="post", qs=""), "POST"): "create_post",
    (URL_PATTERN.format(path="post/\d+", qs=""), "DELETE"): "delete_post"
}

## Log Parsing

In [ ]:
# Parse logs
requests = {"timestamp": [], "method": [], "url": [], "status_code": [], "latency": []}
node_names = os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs"))
for node_name in node_names:
  node_min_timestamp = None
  tarball_path = os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name, "loadgen.tar.gz")
  if os.path.exists(tarball_path):
    with tarfile.open(tarball_path, "r:gz") as tar:
      for filename in tar.getnames():
        if filename.endswith("loadgen.log"):
          with tar.extractfile(filename) as requests_log_file:
            for log in requests_log_file:
              timestamp, method, url, status_code, latency = re.match(REQUEST_LOG_PATTERN, log.decode("utf-8")).groups()
              url = re.sub("limit=\d+&?", "", url)
              url = re.sub("offset=\d+&?", "", url)
              url = re.sub("request_id=[a-zA-Z0-9]+&?", "", url)
              url = re.sub("&$", "", url)
              url = re.sub("\?$", "", url)
              timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
              if node_min_timestamp is None:
                node_min_timestamp = timestamp
              requests["timestamp"].append((timestamp - node_min_timestamp).total_seconds())
              requests["method"].append(method)
              requests["url"].append(url)
              requests["status_code"].append(int(status_code))
              requests["latency"].append(float(latency))

In [ ]:
# Build data frame
requests = pd.DataFrame.from_dict(requests)
requests.sort_values(by="timestamp", ascending=True, inplace=True)
requests["status"] = requests.apply(lambda r: "successful" if r["status_code"] == 200 else "failed", axis=1)
requests["window"] = requests.apply(lambda r: int(r["timestamp"]), axis=1)
requests["type"] = requests.apply(
    lambda r: [request_type for ((pattern, method), request_type) in REQUEST_TO_TYPE.items()
    if method == r["method"] and re.match(pattern, r["url"])][0], axis=1)
requests["rw"] = requests.apply(lambda r: "read" if r["method"] == "GET" else "write", axis=1)

## Status of Requests

In [ ]:
fig = plt.figure(figsize=(18, 6))
ax = fig.gca()
df = requests.groupby(["status"]).count()["method"]
df.plot(ax=ax, kind="pie", title="Number of successful/failed requests", xlabel="", ylabel="", legend=True)

In [ ]:
fig = plt.figure(figsize=(18, 6))
ax = fig.gca()
df = requests[requests["status"] == "failed"].groupby(["status_code"]).count()["method"]
df.plot(ax=ax, kind="pie", title="HTTP status code of failed requests", xlabel="", ylabel="", legend=True)

## Type of Requests

In [ ]:
fig = plt.figure(figsize=(18, 6))
ax = fig.gca()
df = requests.groupby(["rw"]).count()["method"]
df.plot(ax=ax, kind="pie", title="Number of read/write requests", xlabel="", ylabel="", legend=True)

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.grid(alpha=0.75)
df = requests.groupby(["type", "status"]).count()["method"].unstack().fillna(0)
df.plot(ax=ax, kind="bar", stacked=True, title="Number of requests of each type", xlabel="", ylabel="Requests (count)",
        color={"failed": "red", "successful": "blue"}, legend=True, grid=True)

## Request Latency Distribution

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.gca(xlabel="Latency (seconds)", ylabel="Requests (count)")
ax.grid(alpha=0.75)
ax.set_yscale("log")
ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
ax.set_ylim((0, 10000))
ax.set_xticks(range(0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S + 1, (1000 // LATENCY_BIN_IN_MS)))
ax.set_xticklabels([str(s) for s in range(MAX_LATENCY_IN_S + 1)])
df = requests[requests["status"] == "successful"]
df = df[(df["timestamp"] >= RAMP_UP_DURATION) & (df["timestamp"] <= requests["timestamp"].max() - RAMP_DOWN_DURATION)]
df["latency_bin"] = df.apply(lambda r: int(r["latency"] * 1000 // LATENCY_BIN_IN_MS), axis=1)
df["latency_bin"].plot(ax=ax, kind="hist",
    title="Latency Distribution of Successful Requests Excluding Ramping Periods",
    bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)

In [ ]:
fig = plt.figure(figsize=(24, 24))
for (i, (request_type, _)) in enumerate(requests.groupby(["type"])):
  ax = fig.add_subplot(4, 4, i + 1)
  ax.grid(alpha=0.75)
  ax.set_yscale("log")
  ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
  ax.set_ylim((0, 10000))
  ax.set_xticks(range(0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S + 1, (1000 // LATENCY_BIN_IN_MS)))
  ax.set_xticklabels([str(s) for s in range(MAX_LATENCY_IN_S + 1)])
  df = requests[(requests["status"] == "successful") & (requests["type"] == request_type)]
  df = df[(df["timestamp"] >= RAMP_UP_DURATION) & (df["timestamp"] <= requests["timestamp"].max() - RAMP_DOWN_DURATION)]
  df["latency_bin"] = df.apply(lambda r: int(r["latency"] * 1000 // LATENCY_BIN_IN_MS), axis=1)
  df["latency_bin"].plot(ax=ax, kind="hist", title=request_type, xlabel="Latency (seconds)", ylabel="Requests (count)",
      bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)

## Request Latency

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.set_xlim((0, int(requests["timestamp"].max())))
ax.set_ylim((0, 10))
ax.axvline(x=RAMP_UP_DURATION, ls="--", color="green")
ax.axvline(x=requests["timestamp"].max() - RAMP_DOWN_DURATION, ls="--", color="green")
df = requests[requests["status"] == "successful"]
df.set_index("timestamp", inplace=True)
df["latency"].plot(ax=ax, kind="line", title="Latency of Successful Requests", xlabel="Time (seconds)",
    ylabel="Latency (seconds)", color="purple", grid=True, xticks=range(0, int(requests["timestamp"].max()) + 1, 60))

In [ ]:
fig = plt.figure(figsize=(24, 24))
for (i, (request_type, _)) in enumerate(requests.groupby(["type"])):
  ax = fig.add_subplot(4, 4, i + 1)
  ax.grid(alpha=0.75)
  ax.set_xlim((0, int(requests["timestamp"].max())))
  ax.set_ylim((0, 10))
  ax.axvline(x=RAMP_UP_DURATION, ls="--", color="green")
  ax.axvline(x=requests["timestamp"].max() - RAMP_DOWN_DURATION, ls="--", color="green")
  df = requests[(requests["status"] == "successful") & (requests["type"] == request_type)]
  df.set_index("timestamp", inplace=True)
  df["latency"].plot(ax=ax, kind="line", title=request_type, xlabel="Time (seconds)",
      ylabel="Latency (seconds)", color="purple", grid=True, xticks=range(0, int(requests["timestamp"].max()) + 1, 60))

## Request Throughput

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.axvline(x=RAMP_UP_DURATION, ls="--", color="green")
ax.axvline(x=requests["timestamp"].max() - RAMP_DOWN_DURATION, ls="--", color="green")
df = requests.groupby(["window", "status"])["window"].count().unstack().fillna(0)
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
df.plot(ax=ax, kind="line", title="Throughput (requests per second)", xlabel="Time (seconds)",
        ylabel="Requests (count)", color={"failed": "red", "successful": "blue"}, legend=True, grid=True,
        xticks=range(0, int(df.index.max()) + 1, 60))

## Summary

In [ ]:
print("Number of requests")
print("  Total:       %7d" % requests.shape[0])
print("  Status")
print("    Failed:    %7d (%9.5f%%)" % (requests[requests["status"] == "failed"]["status"].count(),
    (requests[requests["status"] == "failed"]["status"].count() / requests.shape[0]) * 100))
print("    Succesful: %7d (%9.5f%%)" % (requests[requests["status"] == "successful"]["status"].count(),
    (requests[requests["status"] == "successful"]["status"].count() / requests.shape[0]) * 100))
print("  Type")
print("    Read:      %7d (%9.5f%%)" % (requests[requests["rw"] == "read"]["rw"].count(),
    (requests[requests["rw"] == "read"]["rw"].count() / requests.shape[0]) * 100))
print("    Write:     %7d (%9.5f%%)" % (requests[requests["rw"] == "write"]["rw"].count(),
    (requests[requests["rw"] == "write"]["rw"].count() / requests.shape[0]) * 100))
print("Experiment duration (s)")
print("  Total:       %7.3f" % requests["timestamp"].max())
print("Latency (ms)")
print("  P99:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.99) * 1000))
print("  P95:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.95) * 1000))
print("  P50:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.50) * 1000))
print("  Avg:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].mean() * 1000))
print("  Std:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].std() * 1000))
print("Throughput (req/s)")
print("  P99:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["window"].max()) + 1), fill_value=0).quantile(0.99))
print("  P95:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["window"].max()) + 1), fill_value=0).quantile(0.95))
print("  P50:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["window"].max()) + 1), fill_value=0).quantile(0.50))
print("  Avg:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["window"].max()) + 1), fill_value=0).mean())
print("  Std:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["window"].max()) + 1), fill_value=0).std())